# Preprocessing the Catholic Bible

In [ ]:
# Importing libraries
import json
import pandas as pd
import re
from utils.os_utils import find_repo_root
import os

# Setting directory
repo_root = find_repo_root()
os.chdir(repo_root)

with open('bible/data/bible_by_theme_preprocessed.json') as f:
    data = json.load(f)

with open('bible/data/bible_v2.json', 'r') as f:
    bible = json.load(f)

In [266]:
# Normalize the data and convert the json in dataframe
data_list = []
for book in bible:
    testamento = book['testamento']
    libro = book['libro']
    titulo = book['titulo']
    for capitulo in book['capitulos']:
        cap_num = capitulo['numero']
        for versiculo in capitulo['versiculos']:
            vers_num = versiculo['numero']
            texto = versiculo['texto']
            data_list.append({
                'Testamento': testamento,
                'Libro': libro,
                'Titulo': titulo,
                'Capitulo': cap_num,
                'Versiculo': vers_num,
                'Texto': texto
            })

# Convert to DataFrame
bible_df = pd.DataFrame(data_list)

In [267]:
bible_df.head()

,Testamento,Libro,Titulo,Capitulo,Versiculo,Texto
0,El Antiguo Testamento,GENESIS,La creación del mundo la caída del hombre,1,1,Al principio Dios creó el cielo y la tierra.
1,El Antiguo Testamento,GENESIS,La creación del mundo la caída del hombre,1,2,"La tierra era algo informe y vacío, las tinieb..."
2,El Antiguo Testamento,GENESIS,La creación del mundo la caída del hombre,1,3,Entonces Dios dijo: «Que exista la luz». Y la ...
3,El Antiguo Testamento,GENESIS,La creación del mundo la caída del hombre,1,4,"Dios vio que la luz era buena, y separó la luz..."
4,El Antiguo Testamento,GENESIS,La creación del mundo la caída del hombre,1,5,y llamó Día a la luz y Noche a las tinieblas. ...


In [268]:
bible_df.drop(columns=['Titulo'], inplace=True)

In [269]:
bible_df[(bible_df["Libro"] == "PRIMERA CARTA A LOS CORINTIOS") & (bible_df["Capitulo"] == 10) &  (bible_df["Versiculo"] == 12)]

,Testamento,Libro,Capitulo,Versiculo,Texto
32598,El Nuevo Testamento,PRIMERA CARTA A LOS CORINTIOS,10,12,"Por eso, el que se cree muy seguro, ¡cuídese d..."


In [270]:
# merged_bible = {}
# for book in bible:
#     book_name = book['libro']
#     merged_bible[book_name] = book["capitulos"]

In [271]:
def get_verse(book, chapter, verse):
    return bible_df[(bible_df["Libro"] == book) & (bible_df["Capitulo"] == chapter) & (bible_df["Versiculo"] == verse)]["Texto"].values[0]

In [272]:
get_verse("PRIMERA CARTA A LOS CORINTIOS", 2, 2)

'Al contrario, no quise saber nada, fuera de Jesucristo, y Jesucristo crucificado.'

In [273]:
data_df_origin = pd.DataFrame(data)
#data_df_origin = data_df_origin[~data_df_origin["pasaje"].isin(["Nahum 1:15", "Malaquías 4:2"])]
#libros =  data_df_origin["pasaje"]

In [274]:
data_df_origin.head()

,pasaje,texto,area_vida
0,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...",[orgullo]
1,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...","[crecimiento espiritual, maternidad, vicio]"
2,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...",[descanso y vacaciones]
3,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,[creación/evolución]
4,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...",[amor]


In [275]:
def parse_biblical_reference(row):
    # Regular expression to capture the book, chapter, and verses
    reference = row["pasaje"]
    pattern = r"(\d*\s*\w+\s*\w*)\s(\d+):([\d,\s-]+)"
    match = re.match(pattern, reference.strip())

    if match:
        libro = match.group(1).strip()
        capitulo = match.group(2).strip()
        versiculos = match.group(3).strip()

        # Expand the verse ranges (e.g., "6-12" to "6,7,8,9,10,11,12")
        versiculo_list = []
        for part in versiculos.split(','):
            part = part.strip()  # Remove surrounding spaces
            if '-' in part:
                start, end = map(int, part.split('-'))
                versiculo_list.extend(range(start, end + 1))
            else:
                versiculo_list.append(int(part))

        versiculos = ",".join(map(str, versiculo_list))

        return {
            "LIBRO": libro,
            "CAPITULO": capitulo,
            "VERSICULOS": versiculos,
            "pasaje": row["pasaje"],
            "texto": row["texto"],
            "area_vida": row["area_vida"]
        }
    else:
        return None

data = []
for i, row in data_df_origin.iterrows():
    parsed_ref = parse_biblical_reference(row)
    if parsed_ref:
        data.append(parsed_ref)

In [276]:
data_df = pd.DataFrame(data)
data_df.head()

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida
0,1 Corintios,10,12,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...",[orgullo]
1,1 Corintios,10,13,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...","[crecimiento espiritual, maternidad, vicio]"
2,1 Corintios,10,31,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...",[descanso y vacaciones]
3,1 Corintios,11,9,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,[creación/evolución]
4,1 Corintios,13,13,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...",[amor]


In [277]:
data_df[data_df["pasaje"] == "Salmos 4:8"]

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida
795,Salmos,4,8,Salmos 4:8,"En paz me acostaré, y asimismo dormiré; Porque...","[protección, descanso y vacaciones]"


In [278]:
# Lista para almacenar los resultados
book_names = []

# Bucle para procesar cada verso
for v in libros:
    # Si comienza con un número
    if v[0].isdigit():
        # Utilizamos split para separar el número del nombre
        parts = v.split(maxsplit=1)
        book_names.append(parts[0] + " " + parts[1].split()[0])  # Añadimos el número y el nombre del libro
    else:
        # Si no comienza con número, simplemente añadimos el nombre del libro
        book_names.append(v.split()[0])

disitnct_book_names = list(set(book_names))
print(disitnct_book_names)

['3 Juan', '1 Reyes', 'Efesios', 'Jeremías', 'Judas', 'Joel', '2 Timoteo', 'Jeremias', 'Exodo', 'Oseas', 'Eclesiastés', 'Romanos', '1 Crónicas', 'Apocalipsis', '2 Colosenses', 'Malaquías', 'Ezequiel', 'Lucas', 'Jonás', 'Isaias', '1 Corintios', 'Isaías', 'Colosenses', 'Marcos', 'Amos', '1 Samuel', 'Juan', '1 Pedro', 'Lamentaciones', 'Tito', '1 Tesalonicenses', 'Job', 'Proverbios', '2 Corintios', '2 Samuel', '1 Juan', '2 Pedro', 'Cantares', 'Zacarías', '1 Timoteo', 'Hebreos', 'Génesis', 'Josúe', 'Salmos', 'Hechos', 'Daniel', 'Levítico', 'Santiago', '2 Crónicas', '2 Reyes', 'Gálatas', 'Mateo', 'Filipenses', 'Nehemías', 'Deuteronomio', 'Números', 'Nahum']


In [279]:
# Original lists
original = ['2 Timoteo', 'Daniel', 'Efesios', 'Eclesiastés', 'Ezequiel', 'Judas', '2 Crónicas', 'Hechos', 'Lucas', 'Tito',
            'Génesis', 'Malaquías', 'Proverbios', 'Nehemías', 'Nahum', '2 Samuel', 'Mateo', '1 Samuel', 'Gálatas',
            '1 Corintios', '2 Reyes', 'Jonás', '1 Timoteo', 'Cantares', 'Deuteronomio', 'Juan', 'Lamentaciones',
            'Romanos', '1 Juan', 'Marcos', '2 Colosenses', 'Hebreos', 'Jeremias', 'Números', '2 Corintios',
            'Zacarías', 'Salmos', 'Josué', 'Santiago', 'Filipenses', 'Jeremías', '1 Pedro', '1 Tesalonicenses', 'Amos',
            '1 Reyes', '3 Juan', 'Oseas', 'Apocalipsis', 'Isaías', 'Job', 'Joel', 'Levítico', '1 Crónicas', 'Colosenses',
            'Exodo', '2 Pedro', 'Isaias', "Josúe"]

replacement =  ["SEGUNDA CARTA A TIMOTEO", "DANIEL", "CARTA A LOS EFESIOS", "ECLESIASTES", "EZEQUIEL", "CARTA DE SAN JUDAS",
                "SEGUNDO LIBRO DE LAS CRONICAS", "HECHOS DE LOS APOSTOLES", "EVANGELIO SEGUN SAN LUCAS", "CARTA A TITO", "GENESIS", "MALAQUIAS", "PROVERBIOS",
                "NEHEMIAS", "NAHUM", "SEGUNDO LIBRO DE SAMUEL", "EVANGELIO SEGUN SAN MATEO", "PRIMER LIBRO DE SAMUEL",
                "CARTA A LOS GALATAS", "PRIMERA CARTA A LOS CORINTIOS", "SEGUNDO LIBRO DE LOS REYES", "JONAS", "PRIMERA CARTA A TIMOTEO",
                "CANTAR DE LOS CANTARES", "DEUTERONOMIO", "EVANGELIO SEGUN SAN JUAN", "LAMENTACIONES", "CARTA A LOS ROMANOS",
                "PRIMERA CARTA DE SAN JUAN", "EVANGELIO SEGUN SAN MARCOS", "SEGUNDA CARTA A LOS TESALONICENSES", "CARTA A LOS HEBREOS",
                "JEREMIAS", "NUMEROS", "SEGUNDA CARTA A LOS CORINTIOS", "ZACARIAS", "SALMOS", "JOSUE", "CARTA DE SANTIAGO",
                "CARTA A LOS FILIPENSES", "JEREMIAS", "PRIMERA CARTA DE SAN PEDRO", "PRIMERA CARTA A LOS TESALONICENSES",
                "AMOS", "PRIMER LIBRO DE LOS REYES", "TERCERA CARTA DE SAN JUAN", "OSEAS", "APOCALIPSIS", "ISAIAS", "JOB",
                "JOEL", "LEVITICO", "PRIMER LIBRO DE LAS CRONICAS", "CARTA A LOS COLOSENSES", "EXODO", "SEGUNDA CARTA DE SAN PEDRO", "ISAIAS",
                "JOSUE"]

# Create a mapping dictionary
mapping = dict(zip(original, replacement))

print(mapping)

{'2 Timoteo': 'SEGUNDA CARTA A TIMOTEO', 'Daniel': 'DANIEL', 'Efesios': 'CARTA A LOS EFESIOS', 'Eclesiastés': 'ECLESIASTES', 'Ezequiel': 'EZEQUIEL', 'Judas': 'CARTA DE SAN JUDAS', '2 Crónicas': 'SEGUNDO LIBRO DE LAS CRONICAS', 'Hechos': 'HECHOS DE LOS APOSTOLES', 'Lucas': 'EVANGELIO SEGUN SAN LUCAS', 'Tito': 'CARTA A TITO', 'Génesis': 'GENESIS', 'Malaquías': 'MALAQUIAS', 'Proverbios': 'PROVERBIOS', 'Nehemías': 'NEHEMIAS', 'Nahum': 'NAHUM', '2 Samuel': 'SEGUNDO LIBRO DE SAMUEL', 'Mateo': 'EVANGELIO SEGUN SAN MATEO', '1 Samuel': 'PRIMER LIBRO DE SAMUEL', 'Gálatas': 'CARTA A LOS GALATAS', '1 Corintios': 'PRIMERA CARTA A LOS CORINTIOS', '2 Reyes': 'SEGUNDO LIBRO DE LOS REYES', 'Jonás': 'JONAS', '1 Timoteo': 'PRIMERA CARTA A TIMOTEO', 'Cantares': 'CANTAR DE LOS CANTARES', 'Deuteronomio': 'DEUTERONOMIO', 'Juan': 'EVANGELIO SEGUN SAN JUAN', 'Lamentaciones': 'LAMENTACIONES', 'Romanos': 'CARTA A LOS ROMANOS', '1 Juan': 'PRIMERA CARTA DE SAN JUAN', 'Marcos': 'EVANGELIO SEGUN SAN MARCOS', '2 Colo

In [280]:
v = []
for i in data_df["LIBRO"]:
    l = mapping.get(i, i)
    v.append(l)
data_df["LIBRO_CAT"] = v
data_df.head()

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida,LIBRO_CAT
0,1 Corintios,10,12,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...",[orgullo],PRIMERA CARTA A LOS CORINTIOS
1,1 Corintios,10,13,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...","[crecimiento espiritual, maternidad, vicio]",PRIMERA CARTA A LOS CORINTIOS
2,1 Corintios,10,31,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...",[descanso y vacaciones],PRIMERA CARTA A LOS CORINTIOS
3,1 Corintios,11,9,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,[creación/evolución],PRIMERA CARTA A LOS CORINTIOS
4,1 Corintios,13,13,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...",[amor],PRIMERA CARTA A LOS CORINTIOS


In [281]:
data_df[data_df["pasaje"] == "1 Samuel 9:27"]

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida,LIBRO_CAT
78,1 Samuel,9,27,1 Samuel 9:27,...mas espera tú un poco para que te declare p...,[descanso en el señor],PRIMER LIBRO DE SAMUEL


In [282]:
vec_verse = []
for i, row in data_df.iterrows():
    libro = row["LIBRO_CAT"]
    capitulo = int(row["CAPITULO"])
    versiculos = row["VERSICULOS"].split(",")
    versiculos = list(map(int, versiculos))
    verse = []
    for versiculo in versiculos:
        # verse.append(get_verse(libro, capitulo, versiculo))
        # result = " ".join(verse)
        try:
            verse.append(get_verse(libro, capitulo, versiculo))
            result = " ".join(verse)
        except:
            print(libro, capitulo, versiculo)
            verse.append("")
    vec_verse.append(result)

data_df["texto_cat"] = vec_verse

MALAQUIAS 4 2
NAHUM 1 15
860
860


In [283]:
data_df[data_df["pasaje"] == "Salmos 4:8"]

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida,LIBRO_CAT,texto_cat
795,Salmos,4,8,Salmos 4:8,"En paz me acostaré, y asimismo dormiré; Porque...","[protección, descanso y vacaciones]",SALMOS,Pero tú has puesto en mi corazón más alegría q...


In [284]:
data_df[data_df["texto_cat"] == "Allí donde oigan el sonido del cuerno, corran a reunirse con nosotros: nuestro Dios combatirá a favor nuestro»."]

,LIBRO,CAPITULO,VERSICULOS,pasaje,texto,area_vida,LIBRO_CAT,texto_cat
568,Nehemías,4,14,Nehemías 4:14,"Después miré, y levantéme, y dije á los princi...",[disciplina/niños],NEHEMIAS,"Allí donde oigan el sonido del cuerno, corran ..."


In [285]:
data_df = data_df.loc[:, ["pasaje", "texto", "texto_cat", "area_vida"]]

In [286]:
data_df[data_df["pasaje"] == "Salmos 4:8"]

,pasaje,texto,texto_cat,area_vida
795,Salmos 4:8,"En paz me acostaré, y asimismo dormiré; Porque...",Pero tú has puesto en mi corazón más alegría q...,"[protección, descanso y vacaciones]"


In [287]:
data_df.head()

,pasaje,texto,texto_cat,area_vida
0,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...","Por eso, el que se cree muy seguro, ¡cuídese d...",[orgullo]
1,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...","Hasta ahora, ustedes no tuvieron tentaciones q...","[crecimiento espiritual, maternidad, vicio]"
2,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...","En resumen, sea que ustedes coman, sea que beb...",[descanso y vacaciones]
3,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,"ni fue creado el hombre a causa de la mujer, s...",[creación/evolución]
4,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...","En una palabra, ahora existen tres cosas: la f...",[amor]


In [289]:
# Convert DataFrame to a dictionary
bible_by_theme_interpreted_json = data_df.to_dict(orient='records')

# Save the list as a JSON file
with open('catholic_bible.json', 'w', encoding='utf-8') as json_file:
    json.dump(bible_by_theme_interpreted_json, json_file, ensure_ascii=False, indent=4)

In [290]:
data_df[data_df["pasaje"] == "Tito 2:15"]

,pasaje,texto,texto_cat,area_vida
856,Tito 2:15,"Esto habla y exhorta, y reprende con toda auto...","Así debes hablar, exhortar y reprender con tod...",[convicción]
